# 3.5 — DeepAR-style Probabilistic Forecast: Price
LSTM with Gaussian output for confidence intervals. 24h ahead, trained on 2015–2017, tested on 2018.

In [1]:
import pandas as pd
import numpy as np
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore')

df = pd.read_parquet('../cleaned_data.parquet')
df['time'] = pd.to_datetime(df['time'], utc=True)

# Use MPS (Apple Silicon GPU) if available
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
try:
    t = torch.randn(2, 2, device=device)
    _ = t @ t
except:
    device = torch.device('cpu')

print(f"Shape: {df.shape}")
print(f"PyTorch: {torch.__version__}")
print(f"Device: {device}")

Shape: (35056, 80)
PyTorch: 2.10.0
Device: mps


In [2]:
# Prepare features: weather + time
target_col = 'price actual'
weather_cols = [
    'pressure_madrid', 'pressure_bilbao', 'pressure_barcelona',
    'pressure_seville', 'pressure_valencia',
    'temp_madrid', 'temp_bilbao', 'temp_barcelona',
    'temp_seville', 'temp_valencia',
    'temp_max_madrid', 'temp_max_bilbao', 'temp_max_barcelona',
    'temp_max_seville', 'temp_max_valencia',
    'temp_min_madrid', 'temp_min_bilbao', 'temp_min_barcelona',
    'temp_min_seville', 'temp_min_valencia',
    'humidity_madrid', 'humidity_bilbao', 'humidity_barcelona',
    'humidity_seville', 'humidity_valencia',
    'wind_speed_madrid', 'wind_speed_bilbao', 'wind_speed_barcelona',
    'wind_speed_seville', 'wind_speed_valencia',
]
time_cols = ['hour', 'month']
feature_cols = weather_cols + time_cols

# Normalize features and target
train_mask = df['time'].dt.year <= 2017

# Compute stats on training data only
target_mean = df.loc[train_mask, target_col].mean()
target_std = df.loc[train_mask, target_col].std()

feat_means = df.loc[train_mask, feature_cols].mean()
feat_stds = df.loc[train_mask, feature_cols].std().replace(0, 1)

# Normalize
target_norm = (df[target_col].values - target_mean) / target_std
features_norm = ((df[feature_cols] - feat_means) / feat_stds).fillna(0).values

# Combine: [target, features] as input channels
all_data = np.column_stack([target_norm, features_norm]).astype(np.float32)

print(f"Input channels: {all_data.shape[1]} (1 target + {len(feature_cols)} features)")
print(f"Target mean: {target_mean:.2f} EUR/MWh, std: {target_std:.2f} EUR/MWh")

Input channels: 33 (1 target + 32 features)
Target mean: 56.03 EUR/MWh, std: 14.33 EUR/MWh


In [3]:
# Dataset: sliding windows
context_length = 168   # 7 days of history
prediction_length = 24  # 24h ahead

class TimeSeriesDataset(Dataset):
    def __init__(self, data, ctx_len, pred_len, start_idx, end_idx):
        self.data = data
        self.ctx_len = ctx_len
        self.pred_len = pred_len
        self.start = start_idx
        self.end = end_idx
    
    def __len__(self):
        return self.end - self.start - self.ctx_len - self.pred_len + 1
    
    def __getitem__(self, idx):
        i = self.start + idx
        x = self.data[i : i + self.ctx_len]
        y = self.data[i + self.ctx_len : i + self.ctx_len + self.pred_len, 0]
        x_future = self.data[i + self.ctx_len : i + self.ctx_len + self.pred_len, 1:]
        return (
            torch.from_numpy(x),
            torch.from_numpy(x_future),
            torch.from_numpy(y),
        )

train_end = int(train_mask.sum())
train_ds = TimeSeriesDataset(all_data, context_length, prediction_length, 0, train_end)
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=0)

print(f"Train samples: {len(train_ds)}")
print(f"Context: {context_length}h, Prediction: {prediction_length}h")

Train samples: 26106
Context: 168h, Prediction: 24h


In [4]:
# DeepAR-style model: LSTM encoder + Gaussian output (mu, sigma)
# Scheduled sampling: during training, randomly use model's own predictions
# instead of ground truth to reduce train/test mismatch
class ProbabilisticLSTM(nn.Module):
    def __init__(self, input_size, future_size, hidden_size, num_layers, pred_len):
        super().__init__()
        self.pred_len = pred_len
        self.encoder = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.1)
        self.decoder_cell = nn.LSTMCell(1 + future_size, hidden_size)
        self.mu_head = nn.Linear(hidden_size, 1)
        self.sigma_head = nn.Linear(hidden_size, 1)
    
    def forward(self, x_context, x_future, y_target=None, tf_ratio=1.0):
        _, (h, c) = self.encoder(x_context)
        h, c = h[-1], c[-1]
        mus, log_sigmas = [], []
        prev_y = x_context[:, -1, 0:1]
        
        for t in range(self.pred_len):
            dec_input = torch.cat([prev_y, x_future[:, t, :]], dim=1)
            h, c = self.decoder_cell(dec_input, (h, c))
            mu = self.mu_head(h)
            log_sigma = self.sigma_head(h)
            mus.append(mu)
            log_sigmas.append(log_sigma)
            
            if y_target is not None and torch.rand(1).item() < tf_ratio:
                prev_y = y_target[:, t:t+1]
            else:
                prev_y = mu.detach()
        
        return torch.cat(mus, dim=1), torch.cat(log_sigmas, dim=1)

input_size = all_data.shape[1]
future_size = len(feature_cols)

model = ProbabilisticLSTM(
    input_size=input_size,
    future_size=future_size,
    hidden_size=64,
    num_layers=2,
    pred_len=prediction_length,
).to(device)

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"On device: {device}")

Model parameters: 84,098
On device: mps


In [5]:
# Train with Gaussian NLL loss + scheduled sampling
# Teacher forcing ratio decays from 1.0 -> 0.3 over training
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=25)

def gaussian_nll(mu, log_sigma, target):
    sigma = torch.exp(log_sigma) + 1e-6
    return torch.mean(0.5 * torch.log(sigma**2) + 0.5 * ((target - mu) / sigma) ** 2)

n_epochs = 25
for epoch in range(n_epochs):
    model.train()
    tf_ratio = max(0.3, 1.0 - epoch / (n_epochs * 0.7))
    losses = []
    for x_ctx, x_fut, y in train_loader:
        x_ctx, x_fut, y = x_ctx.to(device), x_fut.to(device), y.to(device)
        mu, log_sigma = model(x_ctx, x_fut, y, tf_ratio=tf_ratio)
        loss = gaussian_nll(mu, log_sigma, y)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        losses.append(loss.item())
    
    scheduler.step()
    avg_loss = np.mean(losses)
    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {avg_loss:.4f}, TF: {tf_ratio:.2f}, LR: {scheduler.get_last_lr()[0]:.5f}")

print('Training complete')

Epoch 1/25, Loss: -0.5906, TF: 1.00, LR: 0.00100


Epoch 5/25, Loss: -1.3401, TF: 0.77, LR: 0.00090


Epoch 10/25, Loss: -1.2813, TF: 0.49, LR: 0.00065


Epoch 15/25, Loss: -1.1968, TF: 0.30, LR: 0.00035


Epoch 20/25, Loss: -1.2824, TF: 0.30, LR: 0.00010


Epoch 25/25, Loss: -1.2984, TF: 0.30, LR: 0.00000
Training complete


In [6]:
# Generate forecasts on 2018 test data
model.eval()
test_start = train_end
test_end = len(all_data)

all_mu = []
all_sigma = []
all_actuals_list = []
all_times = []

with torch.no_grad():
    for i in range(test_start, test_end - prediction_length, prediction_length):
        if i - context_length < 0:
            continue
        
        x_ctx = torch.from_numpy(all_data[i - context_length : i]).unsqueeze(0).to(device)
        x_fut = torch.from_numpy(all_data[i : i + prediction_length, 1:]).unsqueeze(0).to(device)
        
        mu, log_sigma = model(x_ctx, x_fut, tf_ratio=0.0)
        sigma = torch.exp(log_sigma) + 1e-6
        
        # Denormalize
        mu_val = mu.squeeze().cpu().numpy() * target_std + target_mean
        sigma_val = sigma.squeeze().cpu().numpy() * target_std
        actual_val = all_data[i : i + prediction_length, 0] * target_std + target_mean
        
        times = df['time'].iloc[i : i + prediction_length].values
        
        all_mu.append(mu_val)
        all_sigma.append(sigma_val)
        all_actuals_list.append(actual_val)
        all_times.append(times)

print(f"Generated {len(all_mu)} forecast windows across 2018")

Generated 364 forecast windows across 2018


In [7]:
# Residual correction: train on training data errors to fix systematic biases
from sklearn.ensemble import GradientBoostingRegressor

# Run DeepAR inference on training data (no teacher forcing)
train_mu_list, train_actual_list, train_feat_list = [], [], []
with torch.no_grad():
    for i in range(context_length, train_end - prediction_length, prediction_length):
        x_ctx = torch.from_numpy(all_data[i - context_length : i]).unsqueeze(0).to(device)
        x_fut = torch.from_numpy(all_data[i : i + prediction_length, 1:]).unsqueeze(0).to(device)
        mu, _ = model(x_ctx, x_fut, tf_ratio=0.0)
        train_mu_list.append(mu.squeeze().cpu().numpy() * target_std + target_mean)
        train_actual_list.append(all_data[i : i + prediction_length, 0] * target_std + target_mean)
        train_feat_list.append(all_data[i : i + prediction_length, 1:])

# Train per-horizon GBR correction models
correction_models = []
for h in range(prediction_length):
    X_h = np.array([np.append(f[h], m[h]) for f, m in zip(train_feat_list, train_mu_list)])
    y_h = np.array([a[h] - m[h] for a, m in zip(train_actual_list, train_mu_list)])
    gbr = GradientBoostingRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42)
    gbr.fit(X_h, y_h)
    correction_models.append(gbr)

# Apply correction to test predictions
corrected_mu = []
for i, mu_val in enumerate(all_mu):
    idx_start = test_start + i * prediction_length
    feats = all_data[idx_start : idx_start + prediction_length, 1:]
    corrected = np.array([mu_val[h] + correction_models[h].predict(np.append(feats[h], mu_val[h]).reshape(1, -1))[0]
                          for h in range(prediction_length)])
    corrected_mu.append(corrected)

raw_mae = np.mean([np.mean(np.abs(a - m)) for a, m in zip(all_actuals_list, all_mu)])
corr_mae = np.mean([np.mean(np.abs(a - c)) for a, c in zip(all_actuals_list, corrected_mu)])

# Use correction only if it actually improves MAE
if corr_mae < raw_mae:
    final_mu = corrected_mu
    final_mae = corr_mae
    print(f"Correction helps! Raw: {raw_mae:.2f} -> Corrected: {corr_mae:.2f} EUR/MWh ({(1-corr_mae/raw_mae)*100:.1f}%)")
else:
    final_mu = list(all_mu)
    final_mae = raw_mae
    print(f"Correction doesn't help (Raw: {raw_mae:.2f}, Corrected: {corr_mae:.2f}). Using raw predictions.")

# Multi-quantile calibration (gradient fan chart)
quantile_levels = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 95]
errors_by_horizon = [[] for _ in range(prediction_length)]
for pred, actual in zip(final_mu, all_actuals_list):
    for h in range(prediction_length):
        errors_by_horizon[h].append(actual[h] - pred[h])

quantile_offsets = {q: np.array([np.percentile(errors_by_horizon[h], q) for h in range(prediction_length)]) for q in quantile_levels}

all_forecasts = []
for pred in final_mu:
    fc = {'p50': pred}
    for q in quantile_levels:
        fc[f'p{q}'] = pred + quantile_offsets[q]
    all_forecasts.append(fc)

cov_80 = np.mean([np.mean((a >= fc['p10']) & (a <= fc['p90'])) for a, fc in zip(all_actuals_list, all_forecasts)])
cov_90 = np.mean([np.mean((a >= fc['p5']) & (a <= fc['p95'])) for a, fc in zip(all_actuals_list, all_forecasts)])

tso_col = 'price day ahead'
tso_maes = [np.mean(np.abs(all_actuals_list[i] - df[tso_col].iloc[test_start + i*prediction_length : test_start + i*prediction_length + prediction_length].values))
            for i in range(len(all_actuals_list))]

print(f"Final MAE:      {final_mae:.2f} EUR/MWh")
print(f"TSO MAE:        {np.mean(tso_maes):.2f} EUR/MWh")
print(f"80% coverage:   {cov_80*100:.1f}%")
print(f"90% coverage:   {cov_90*100:.1f}%")

Correction doesn't help (Raw: 7.21, Corrected: 8.47). Using raw predictions.
Final MAE:      7.21 EUR/MWh
TSO MAE:        8.87 EUR/MWh
80% coverage:   79.7%
90% coverage:   89.6%


In [8]:
# Export sample week with multi-quantile data for gradient fan chart
sample_windows = range(9, 16)

sample_data = []
for w in sample_windows:
    if w >= len(all_forecasts):
        break
    fc = all_forecasts[w]
    actual = all_actuals_list[w]
    times = all_times[w]
    
    for h in range(prediction_length):
        t = pd.Timestamp(times[h])
        point = {
            'time': t.strftime('%Y-%m-%d %H:%M'),
            'actual': round(float(actual[h]), 2),
        }
        for q in quantile_levels:
            point[f'p{q}'] = round(float(fc[f'p{q}'][h]), 2)
        sample_data.append(point)

print(f"Sample data: {len(sample_data)} hours")
print(f"Quantiles exported: {[f'p{q}' for q in quantile_levels]}")

Sample data: 168 hours
Quantiles exported: ['p5', 'p10', 'p20', 'p30', 'p40', 'p50', 'p60', 'p70', 'p80', 'p90', 'p95']


In [9]:
# Export JSON
import os
os.makedirs('../dashboard/public/data', exist_ok=True)

output = {
    'target': 'price',
    'model': 'DeepAR + GBR Correction' if corr_mae < raw_mae else 'DeepAR (LSTM + Gaussian)',
    'prediction_length_hours': prediction_length,
    'context_length_hours': context_length,
    'metrics': {
        'mae': round(final_mae, 2),
        'tso_mae': round(float(np.mean(tso_maes)), 2),
        'coverage_80': round(cov_80 * 100, 1),
        'coverage_90': round(cov_90 * 100, 1),
    },
    'quantile_levels': quantile_levels,
    'sample_forecast': sample_data,
}

with open('../dashboard/public/data/deepar_price.json', 'w') as f:
    json.dump(output, f, indent=2)

print('Saved deepar_price.json')
print(f"MAE:          {output['metrics']['mae']} EUR/MWh")
print(f"TSO MAE:      {output['metrics']['tso_mae']} EUR/MWh")
print(f"80% Coverage: {output['metrics']['coverage_80']}%")
print(f"90% Coverage: {output['metrics']['coverage_90']}%")

Saved deepar_price.json
MAE:          7.21 EUR/MWh
TSO MAE:      8.87 EUR/MWh
80% Coverage: 79.7%
90% Coverage: 89.6%
